## Code to Generate TIF from Google Earth Engine
### Requirements:
- Google Earth Engine Project
- Google Drive access (to store the accuracy metrics in a csv)
- Google drive access (to store the generated TIFs)
- A `coordinates.xlsx` file - to copy the coordinates from

### Number of classes - 4
`{Water, Building, Sparse Vegetation, Dense Vegetation}`

### Initialize

In [1]:
import ee
import csv
import os

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='omkarsoak-ee')

In [2]:
from google.colab import drive
drive.mount('/content/drive')
csv_filename = '/content/drive/My Drive/BTechProject/Eur_new_classification_results1.csv'

Mounted at /content/drive


### COMBINED RGB + MASK SVM

In [3]:
def process_point_svm(lon, lat, city, state, year, filename):
    # Define the center point
    center = ee.Geometry.Point([lon, lat])

    # Define the bounding box with a buffer radius (2560 meters for 512x512 pixels at 10m resolution)
    geometry = center.buffer(2560).bounds()

    # Load Sentinel-2 MSI Level-2A as ImageCollection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(geometry)
                 .filterDate(f'{year}-07-01', f'{year}-09-30')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
                 .median())

    # Calculate indices and masks
    ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # ndwi = sentinel2.normalizedDifference(['B3', 'B8']).rename('NDWI')
    swm = sentinel2.expression(
        '(B2 + B3) / (B8 + B11)',
        {
            'B2': sentinel2.select('B2'),
            'B3': sentinel2.select('B3'),
            'B8': sentinel2.select('B8'),
            'B11': sentinel2.select('B11')
        }
    ).rename('SWM')

    # Combine NDVI and NDWI into a single image for sampling
    feature_image = ndvi.addBands(swm)

    # Define class masks
    water_mask = swm.gt(1.5)
    building_mask = ndvi.gt(0.0).And(ndvi.lt(0.4))
    sparse_veg_mask = ndvi.gte(0.4).And(ndvi.lt(0.7))
    dense_veg_mask = ndvi.gte(0.7)

    # Sample points for each class
    NUM_OF_PIXELS = 50000

    def sample_points(mask, class_value):
        return feature_image.updateMask(mask).sample(
            region=geometry,
            scale=10,
            numPixels=NUM_OF_PIXELS,
            geometries=True
        ).map(lambda f: f.set('class', class_value))

    water_points = sample_points(water_mask, 0)
    building_points = sample_points(building_mask, 1)
    sparse_veg_points = sample_points(sparse_veg_mask, 2)
    dense_veg_points = sample_points(dense_veg_mask, 3)

    # Merge all training points
    training_points = water_points.merge(building_points).merge(sparse_veg_points).merge(dense_veg_points)

    # Split the data into training (80%) and validation (20%) sets
    with_random = training_points.randomColumn()
    split = 0.8
    training_data = with_random.filter(ee.Filter.lt('random', split))
    validation_data = with_random.filter(ee.Filter.gte('random', split))

    # Train an SVM classifier
    svm_classifier = ee.Classifier.libsvm().train(
        features=training_data,
        classProperty='class',
        inputProperties=['NDVI', 'SWM']
    )

    # Classify the image
    classified_image = feature_image.classify(svm_classifier)
    # classified_image = classified_image.toUint16()

    # # Evaluate the classifier using the validation data
    # validation = validation_data.classify(svm_classifier)
    # validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    # print('Validation Error Matrix:', validation_confusion_matrix.getInfo())
    # print('Validation Overall Accuracy:', validation_confusion_matrix.accuracy().getInfo())

    # # Compute training accuracy
    # training_confusion_matrix = svm_classifier.confusionMatrix()
    # print('Training Error Matrix:', training_confusion_matrix.getInfo())
    # print('Training Overall Accuracy:', training_confusion_matrix.accuracy().getInfo())


    # Evaluate the classifier using the validation data
    validation = validation_data.classify(svm_classifier)
    validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    validation_accuracy = validation_confusion_matrix.accuracy().getInfo()

    # Compute training accuracy
    training_confusion_matrix = svm_classifier.confusionMatrix()
    training_accuracy = training_confusion_matrix.accuracy().getInfo()

    # # Save results to CSV
    # csv_filename = "classification_results.csv"
    header = ["City", "State", "Year", "Filename",
              "Training Accuracy", "Training Confusion Matrix",
              "Validation Accuracy", "Validation Confusion Matrix"]

    data = [
        city, state, year, filename,
        training_accuracy, training_confusion_matrix.getInfo(),
        validation_accuracy, validation_confusion_matrix.getInfo()
    ]

    # Write to CSV (append if file exists, otherwise create)
    try:
        with open(csv_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:  # File is empty, write header
                writer.writerow(header)
            writer.writerow(data)
    except Exception as e:
        print(f"Error writing to CSV: {e}")

    # print(f"Results saved to {csv_filename}.")

    # try:
    #   file_exists = os.path.isfile(csv_filename)
    #   with open(csv_filename, 'a', newline='') as file:
    #       writer = csv.writer(file)
    #       if not file_exists:  # If the file doesn't exist, write the header
    #           writer.writerow(header)
    #       writer.writerow(data)
    # except Exception as e:
    #   print(f"Error writing to CSV: {e}")

    print(f"Results saved to {csv_filename}.")

    # Enhanced RGB export
    enhanced_rgb = sentinel2.select(['B4', 'B3', 'B2']).multiply(2.0)
    viz_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }

    # Export tasks
    # 1. Enhanced RGB
    enhanced_rgb_task = ee.batch.Export.image.toDrive(
        image=enhanced_rgb.uint16().clip(geometry).visualize(**viz_params),
        description=f'EnhancedRGB_{filename}_{year}',
        folder='ChangeDetectionEur',
        fileNamePrefix=f'{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # 2. Classified Image
    class_vis = {
        'min': 0,
        'max': 3,
        'palette': ['lightblue', 'white', 'lightgreen', 'darkgreen']
    }
    mask_task = ee.batch.Export.image.toDrive(
        image=classified_image.uint16(),
        description=f'm_{filename}_{year}',
        folder='ChangeDetectionEur',
        fileNamePrefix=f'm_{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # Start both export tasks
    enhanced_rgb_task.start()
    mask_task.start()

    print(f"Tasks started for {city} at ({lon}, {lat}).")

### Import data from csv
- directly copy paste into a cell
- just do `data = pd.read_csv(...`

In [4]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
Netherlands,Rotterdam,2019,2024,"4.47777963162828,51.9222407862643",51.92224079,4.477779632,"4.47777963162828,51.9722407862643","4.52777963162828,51.9722407862643","4.52777963162828,51.9222407862643","4.52777963162828,51.8722407862643","4.47777963162828,51.8722407862643","4.42777963162828,51.8722407862643","4.42777963162828,51.9222407862643","4.42777963162828,51.9722407862643"
'''), header=None)

### Run

In [5]:
coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
ncities = data.shape[0]

# Process each city
for j in range(ncities):
    cityData = data.iloc[j]
    city = cityData[1]
    state = cityData[0]
    print(city, state)

    for i in range(9):
        try:
            coord = cityData.iloc[coord_list[i]].split(',')
            filename = f'{state}_{city}_{coord_name[i]}'
            print(filename)

            lon = float(coord[0])
            lat = float(coord[1])
            process_point_svm(lon, lat, city, state, '2019', filename)
            process_point_svm(lon, lat, city, state, '2024', filename)
        except Exception as e:
            print(f"Error processing {state}, {city}, {coord_name[i]}: {e}")
            print(f"Exception details: {type(e)._name_} - {e}")
    print()

print("All tasks have been started.")

Rotterdam Netherlands
Netherlands_Rotterdam_C
Results saved to /content/drive/My Drive/BTechProject/Eur_new_classification_results1.csv.
Tasks started for Rotterdam at (4.47777963162828, 51.9222407862643).
Results saved to /content/drive/My Drive/BTechProject/Eur_new_classification_results1.csv.
Tasks started for Rotterdam at (4.47777963162828, 51.9222407862643).
Netherlands_Rotterdam_N
Results saved to /content/drive/My Drive/BTechProject/Eur_new_classification_results1.csv.
Tasks started for Rotterdam at (4.47777963162828, 51.9722407862643).
Results saved to /content/drive/My Drive/BTechProject/Eur_new_classification_results1.csv.
Tasks started for Rotterdam at (4.47777963162828, 51.9722407862643).
Netherlands_Rotterdam_NE


KeyboardInterrupt: 

### Run (without Error handling)

In [ ]:
coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
ncities = data.shape[0]
# Process each city
for j in range(ncities):
  cityData = data.iloc[j]
  city = cityData[1]
  state = cityData[0]
  # print(cityData)
  print(city, state)
  for i in range(9):
      coord = cityData.iloc[coord_list[i]].split(',')
      filename = f'{state}_{city}_{coord_name[i]}'
      print(filename, end=';')

      lon = float(coord[0])
      lat = float(coord[1])
      print(lon, lat)
      # process_point(lon, lat, city, state, '2019', filename)
      # process_point(lon, lat, city, state, '2024', filename)
      process_point_svm(lon, lat, city, state, '2019', filename)
      process_point_svm(lon, lat, city, state, '2024', filename)
  print()

print("All tasks have been started.")